# Testing LB on FABRIC U280

This notebook helps sets up a sender and multiple receiver nodes on FABRIC such that they can communicate with a LB provisioned in FABRIC. Unlike the 'FABRIC-live-lb-tester' notebook, this one does NOT use FABNetv4Ext service - only FABNetv4, consequently the security measures are more limited, since none of this is exposed to the internet. One of the nodes is designated as a sender and the rest as worker-receivers.

See the following diagram:

<div>
    <img src="figs/live-lb.png" width=500>
</div>


## Preamble

This code should *always* be executed regardless of whether you are starting a new slice or returning to an existing slice.

In [ ]:
#
# EDIT THIS
#

# GitHub SSH key file (private) registered using the GitHubSSH.ipynb notebook referenced above
github_key = '/home/fabric/work/fabric_confi/github_ecdsa'

# Note for best management network IPv4 connectivity pick from
# 'UCSD', 'SRI', 'FIU' or 'TOKY' - these sites have
# IPv4. Other sites use IPv6 management and have trouble
# retrieving git-lfs artifacts.

# site_list_override = None

# if you want to force a site list instead of using random
#site_list_override = ['SRI', 'UCSD', 'CLEM']

# (super)core sites - should be low loss
site_list_override = ['SALT', 'KANS', 'NEWY', 'WASH', 'LOSA', 'DALL', 'ATLA']

# high capacity sites (may have losses at high bandwidth)
# site_list_override = ['STAR', 'INDI', 'NCSA', 'TACC', 'UCSD', 'PSC']

# these we always exclude
site_exclude_list = ['EDUKY', 'EDC']

# how many workers do we want? (in addition to one sender)
number_of_workers = 3

# base distro 'ubuntu2[012]' or 'rocky[89]'
distro_name = 'ubuntu22'
distro_version = distro_name[-2:]

# map from distro to image name
images = {
    'ubuntu20': 'default_ubuntu_20',
    'ununtu21': 'default_ubuntu_21',
    'ubuntu22': 'default_ubuntu_22',
    'rocky8': 'default_rocky_8',
    'rocky9': 'default_rocky_9',
}

# note that the below is distribution specific ('ubuntu' for ubuntu and so on)
home_location = {
    'ubunt': '/home/ubuntu',
    'rocky' : '/home/rocky'
}[distro_name[:5]]

vm_key_location = f'{home_location}/.ssh/github_ecdsa'

# worker dimensions
node_attribs = {
    'cores': 8,
    'disk': 100,
    'ram': 24,
    'image': images[distro_name]
}

# slice name
slice_name = f'{number_of_workers + 1}-node U280 LB Tester Slice using {distro_name} 1'

# url of e2sar deps. Find the appropriate version for the OS at https://github.com/JeffersonLab/E2SAR/releases
e2sar_branch = "main"
static_release_url = 'https://github.com/JeffersonLab/E2SAR/releases/download/' # don't need to change this
e2sar_dep_artifcat = 'e2sar-deps_0.1.4_amd64.deb'
e2sar_release_ver = 'E2SAR-main-0.1.4'
e2sar_dep_url = static_release_url + e2sar_release_ver + "-" + distro_name[:-2] + "-" + distro_version + ".04/" + e2sar_dep_artifcat

# this is the IPv4 address and Control Plane listening port of the LB node created by the other notebook
lb_node_ip = "10.138.47.254"
lb_cp_port = 18008
# the admin token is also generated by the U280 LB notebook
lb_admin_token = "0DTIKKNEYMMOUP283AP8UM4ABI45PNJ3"

ejfat_admin_uri = f"ejfats://{lb_admin_token}@{lb_node_ip}:{lb_cp_port}/"

#
# SHOULDN'T NEED TO EDIT BELOW
#
# Preamble
import json
import time
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

# gets prepended to site name - this network is per site
net_name_prefix = 'fabnetv4'

# this is the NIC to use
nic_model = 'NIC_Basic'

def execute_single_node(node, commands):
    for command in commands:
        print(f'\tExecuting "{command}" on node {node.get_name()}')
        #stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        stdout, stderr = node.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')
        
def execute_commands(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node(n, commands)
    else:
        execute_single_node(node, commands)

def execute_single_node_on_thread(node, commands):
    # concatenate the commands using ';' and execute
    allcommands = ';'.join(commands)
    node.execute_thread(allcommands, output_file=node.get_name() + '_thread.log')

def execute_commands_on_threads(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node_on_thread(n, commands)
    else:
        execute_single_node_on_thread(node, commands)

def make_node_name(site_name, node_idx):
    return '_'.join([f"Worker{node_idx}", site_name])

def make_net_name(site_name):
    return '_'.join([net_name_prefix, site_name])

# return slice with one node on one site
def starter_slice(site_name):
    #node_name = make_node_name(site_name, 1)
    node_name = '_'.join(["Sender", site_name])
    net_name = make_net_name(site_name)

    slice = fablib.new_slice(name=slice_name)
    node = slice.add_node(name=node_name, site=site_name, **node_attribs)

    # postboot configuration is under 'post-boot' directory
    node.add_post_boot_upload_directory('post-boot','.')
    node.add_post_boot_execute(f'chmod +x post-boot/sender.sh && ./post-boot/sender.sh')
    
    # attach to network
    nic_interface = node.add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
    net = slice.add_l3network(name=net_name, interfaces=[nic_interface], type='IPv4')

    return slice

def add_node_to_slice(site_name, node_idx, inc, slice):

    net_name = make_net_name(site_name)

    while inc > 0:
        node_name = make_node_name(site_name, node_idx)
        node_idx += 1
        
        node = slice.add_node(name=node_name, site=site_name, **node_attribs)
    
        # postboot configuration is under 'post-boot' directory
        node.add_post_boot_upload_directory('post-boot','.')
        node.add_post_boot_execute(f'chmod +x post-boot/recver.sh && ./post-boot/recver.sh')
    
        nic_interface = node.add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
        
        # attach to a network, create network if needed
        net = slice.get_network(name=net_name)
        if net is None:
            net = slice.add_l3network(name=net_name, type='IPv4')
            
        net.add_interface(nic_interface)
        inc -= 1

    return None

def check_modify(slice, selected_site_list, nodes_in_slice, expected_to_add):

    success = True
    idx = 1
    while(expected_to_add >= idx):
        # find sliver reservation for new node
        node_sliver = slice.list_slivers(fields=['name', 'state'], 
                                         filter_function=lambda x: x['type'] == 'node' and 
                                             x['name'] == make_node_name(selected_site_list[0], nodes_in_slice + idx) and 
                                             x['state'] == 'Active')
        # if it is none - it failed
        if node_sliver is None:
            success = False
            break
        else:
            idx += 1

    return success

# until fablib fixes this
def get_management_os_interface(node) -> str or None:
        """
        Gets the name of the management interface used by the node's
        operating system. 

        :return: interface name
        :rtype: String
        """
        stdout, stderr = node.execute("sudo ip -j route list", quiet=True)
        stdout_json = json.loads(stdout)

        for i in stdout_json:
            if i["dst"] == "default":
                return i["dev"]

        stdout, stderr = node.execute("sudo ip -6 -j route list", quiet=True)
        stdout_json = json.loads(stdout)

        for i in stdout_json:
            if i["dst"] == "default":
                return i["dev"]

        return None

## Helpers

If you ever forget which images are available, run this cell:

In [ ]:
# List available images (this step is optional)
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

## Prepare to create a new slice (skip if exists)

In [ ]:
# list all slices I have running
output_dataframe = fablib.list_slices(output='pandas')
if output_dataframe:
    print(output_dataframe)
else:
    print('No active slices under this project')

In [ ]:
# Identify sites in continental US we want to use (NOOP if override is set)
lon_west=-124.3993243
lon_east=-69.9721573
candidate_sites = 7
free_nodes_worth = 3 # how many nodes worth are we looking per site

# get a list of random sites, avoiding thos on the exclude list
# unless there is an override
if site_list_override is None:
    selected_site_list = fablib.get_random_sites(count=candidate_sites, avoid=site_exclude_list,
                                            filter_function=lambda x: x['location'][1] < lon_east
                                            and x['location'][1] > lon_west 
                                            and x['cores_available'] > free_nodes_worth * node_attribs['cores']
                                            and x['ram_available'] > free_nodes_worth * node_attribs['ram'] 
                                            and x['disk_available'] > free_nodes_worth * node_attribs['disk']) 
else:
    selected_site_list = site_list_override

if selected_site_list:
    print(f'Selected sites are {selected_site_list}')
else:
    print('Unable to find a sites matching the requirements')


## Create slice iteratively (skip if exists)

We may or may not get all the nodes we want immediately - we use iteration with slice modify to get to the max/desired number of nodes across the selected sites.

### Create Starter Slice

In [ ]:
# we start by establishing a slice with one sender node at some site, we keep track which sites we failed 
# and don't try those again

keep_trying = True
succeeded = False

site_list_iter = iter(selected_site_list)
failed_sites = {}
site_name = None

while keep_trying:

    try:
        site_name = next(site_list_iter)
        print(f'Trying site {site_name} from {selected_site_list}')
        
        # define a starter slice
        slice = starter_slice(site_name)

        print(f'Submitting starter slice "{slice_name}" with sender on site {site_name}')
        slice_id = slice.submit()

        # check the state of this slice
        slices = fablib.get_slices(excludes=[], slice_id=slice_id)
        if slices[0].get_state() == 'Dead':
            print(f'Failed on site {site_name}, proceeding')
        else:
            print(f'Succeeded on site {site_name} with state {slices[0].get_state()}')
            keep_trying = False
            succeeded = True
    except StopIteration: 
        print('No more sites to look at, exiting')
        keep_trying = False
    except Exception as e:
        print(f'Unexpected exception {e}, exiting')
        keep_trying = False

if succeeded:
    print(f'Succeeded in creating a slice on {site_name}, will avoid sites {failed_sites}')
    selected_site_list = list(filter(lambda x: x not in failed_sites, selected_site_list))
    print(f'Proceeding with sites {selected_site_list}')

### Modify the Slice to add Workers

Now that the base with the sender slice is created we will iteratively add workers on sites one at a time using first-fit policy until we get to the desired number of workers or run out of sites.

In [ ]:
remaining_workers = number_of_workers
node_idx = 1
node_increment = 3
nodes_in_slice = 0 # we don't count sender in this case

while remaining_workers > 0 and len(selected_site_list) > 0:
    slice = fablib.get_slice(name=slice_name)
    
    try:
        site_name = selected_site_list[0]
        print(f'There are {remaining_workers} remaining workers to create. Trying site {site_name} from {selected_site_list}')
        expected_to_add = node_increment if remaining_workers >= node_increment else remaining_workers
        add_node_to_slice(site_name, node_idx, expected_to_add, slice)
        
        print(f'Submitting slice modification to "{slice_name}" to add {expected_to_add} nodes for site {site_name}')
        slice_id = slice.modify()
        
        # check the state of this slice
        slice = fablib.get_slice(name=slice_name)

        if check_modify(slice, selected_site_list, nodes_in_slice, expected_to_add):
            print(f'Succeeded adding {expected_to_add} nodes on site {site_name}.')
            # successfully provisioned
            node_idx += expected_to_add
            remaining_workers -= expected_to_add
            nodes_in_slice += expected_to_add
        else:
            print(f'Failed to provision on site {site_name}.')
            # this site is full, moving on
            selected_site_list.remove(site_name)            
    except Exception as e:
        remaining_workers = -1
        print(f'Unexpected exception {e}, exiting')
        break

if remaining_workers == 0:
    print('Succeeded in creating all workers')
else:
    print(f'Unable to create {remaining_workers}')


## Get Slice Details (always execute)

The following code sets up data structures so all the follow up cells work properly. Execute it regardless of whether you just created the slice or coming back to an existing slice.

In [ ]:
def find_net(net_list, name):
    return next(filter(lambda x: x.get_name() == name, net_list), None)

# this is the full /10 routable FABRIC IPv4
full_subnet = ipaddress.IPv4Network('10.128.0.0/10')

# get slice details 
slice = fablib.get_slice(name=slice_name)

a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

# arrange nodes and network services by site for future convenience
net_objects = slice.get_networks()
node_objects = slice.get_nodes()

slivers_by_site = dict()

print('Arranging nodes and networks by site and getting available IP addresses')
for node in node_objects:
    node_site = node.get_site()
    if not slivers_by_site.get(node_site):
        slivers_by_site[node_site] = dict()
        slivers_by_site[node_site]['nodes'] = set()
        slivers_by_site[node_site]['net'] = find_net(net_objects, make_net_name(node_site))
    slivers_by_site[node_site]['nodes'].add(node)

print('Listing IPv4 addresses per service (per site)')
for net in net_objects:
    print(f'{net.get_name()} has {net.get_subnet()}')


## Perform network configuration

### Set up routing

In [ ]:
def get_valid_ips(ip_list, gateway, qty=1):
    # make sure the returned IP isn't the gateway
    ips = []
    while qty > 0:
        ip = ip_list.pop()
        while ip == gateway:
            ip = ip_list.pop()
        ips.append(ip)
        qty -= 1

    return ips

# allocate IP addresses in each site network services
print('Allocating IP addresses in sites where slice is present')
for site_name, site_slivers  in slivers_by_site.items():
    print(f'Processing {site_name}')
    site_net = site_slivers['net']
    site_nodes = site_slivers['nodes']
    # this properly lists needed available IPs for this subnet
    site_slivers['ips'] = list(site_net.get_subnet().hosts())[:len(site_nodes)]
    site_slivers['ips'] = get_valid_ips(list(site_net.get_subnet().hosts()), site_net.get_gateway(), len(site_nodes))
    print(f'Using the following IPs: {site_slivers["ips"]}')

In [ ]:
# configure node interfaces with these IP addresses
print('Configuring interfaces on nodes')
for site_name, site_slivers in slivers_by_site.items():
    print(f'Processing {site_name}')
    site_net = site_slivers['net']
    site_nodes = site_slivers['nodes']
    site_addrs = site_slivers['ips']
    for node, addr in zip(site_nodes, site_addrs):
        print(f'  Adding address {addr} to node {node.get_name()} in subnet {site_net.get_subnet()}')
        # make sure the interface is UP (in rare cases comes up in DOWN state)
        node_iface = node.get_interface(network_name=site_net.get_name())
        execute_single_node(node, [f'sudo ip link set {node_iface.get_os_interface()} up'])
        node_iface.ip_addr_add(addr=addr, subnet=site_net.get_subnet())
        node.ip_route_add(subnet=full_subnet, gateway=site_net.get_gateway())


In [ ]:
# We need to setup the firewall to allow UDP traffic to pass between the nodes
# we do this by moving all interfaces (loopback, management and data) to the 'trusted' zone
# since we are in FABNetv4 we can keep things simple

for site_name, site_slivers in slivers_by_site.items():
    print(f'Processing {site_name}')
    site_net = site_slivers['net']
    for node in site_slivers['nodes']:
        mgmt_iface_name = get_management_os_interface(node)
        data_iface = node.get_interface(network_name=site_net.get_name())
        data_iface_name = data_iface.get_os_interface()
        commands = [
            f'sudo firewall-cmd --permanent --zone=trusted --add-interface={data_iface_name}',
            f'sudo firewall-cmd --permanent --zone=trusted --add-interface=lo',
            f'sudo firewall-cmd --permanent --zone=trusted --add-interface={mgmt_iface_name}',
            f'for i in $(sudo firewall-cmd --zone=public --list-services); do sudo firewall-cmd --zone=public --permanent --remove-service=$i; done',
        ]
        commands.append(f'sudo firewall-cmd --reload')
        commands.append(f'sudo firewall-cmd --list-all --zone=trusted')
        execute_commands([node], commands)

## Tune Buffers and MTUs

In order to have good performance we need to
- Make the UDP send/receive socket buffer size limit larger (applications are assumed to know how to make their buffers larger up to this limit)
- Set MTU to 9k and test with DF=0 ping

In [ ]:
# setup UDP socket buffer sizes to 512M
commands = [
    f"sudo sysctl net.core.rmem_max=536870912",
    f"sudo sysctl net.core.wmem_max=536870912",
    f"sysctl net.core.wmem_max net.core.rmem_max"
]
# walk the nodes
for site_name, site_slivers in slivers_by_site.items():
    for node in site_slivers['nodes']:
        execute_single_node(node, commands)

In [ ]:
# set 9k MTU on dataplane interfaces
mtu=9000

for site_name, site_slivers in slivers_by_site.items():
    site_net = site_slivers['net']
    for node in site_slivers['nodes']:
        data_iface = node.get_interface(network_name=site_net.get_name())
        data_iface_name = data_iface.get_os_interface()
        execute_single_node(node, [f"sudo ip link set dev {data_iface_name} mtu {mtu}"])

In [ ]:
# run a test from every node to the LB node
for site_name, site_slivers in slivers_by_site.items():
    for node in site_slivers['nodes']:
        print(f'Node {node.get_name()} pinging {lb_node_ip}')
        #execute_single_node(node, [f"sudo ping -q -f -s 8972 -c 100 -M do {lb_node_ip}"])
        execute_single_node(node, [f"sudo ping -q -f -c 100 {lb_node_ip}"])

## Customize Nodes

Customize node setup by adding E2SAR installation

### Add E2SAR software

In [ ]:
# install github ssh key and set up build environment variables for interactive logins
commands = [
    f"chmod go-rwx {vm_key_location}",
    # Meson won't detect boost by merely setting cmake_prefix_path, instead set BOOST_ROOT env variable 
    # for gRPC it is enough to set -Dpkg_config_path option to meson
    f"echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib' >> ~/.profile",
    f"echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib' >> ~/.bashrc",
]

for node in slice.get_nodes():    
    # upload the GitHub SSH key onto the VM
    result = node.upload_file(github_key, vm_key_location)
    execute_commands(node, commands)

In [ ]:
# download boost and grpc dependencies from releases
commands = [
    f"wget -q -O boost_grpc.deb {e2sar_dep_url}",
    #f"sudo apt -yq install ./boost_grpc.deb",
    f"sudo dpkg -i ./boost_grpc.deb"
]
 
execute_commands_on_threads(slice.get_nodes(), commands)
print("Note this is executed in parallel on threads, check the log files for completion")

In [ ]:
# checkout E2SAR (including the righte branch) using that key, install grpc and boost binary that is stored in the repo
branch_override=None
if branch_override is not None:
    branch = branch_override
else:
    branch = e2sar_branch
    
commands = [
    f"rm -rf E2SAR",
    f"GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b {branch} git@github.com:JeffersonLab/E2SAR.git",
]
 
execute_commands(slice.get_nodes(), commands)

In [ ]:
# compile and test E2SAR code
# note that most live tests only need the simplest URI - ejfats://token@ip:port/
# however the e2sar_reas_live_test requires data and sync addresses, and data address must
# be real (so we use loopback). Hence the long form of the URI for live tests 
# (other tests simply ignore the parts of the URI they don't need.)

commands = [
    f"cd E2SAR; rm -rf build; PATH=$HOME/.local/bin:/usr/local/bin:$PATH BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib/ meson setup -Dpkg_config_path=/usr/local/lib/pkgconfig/:/usr/lib/lib64/pkgconfig/ --prefix {home_location}/e2sar-install build && sed -i 's/-std=c++11//g' build/build.ninja",
    f"PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/  meson compile -j 8 -C build",
    f"PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/  meson test --suite unit  --timeout 0 -C build",
]

# NOTE THIS EXECUTES ON THREADS IN PARALLEL, CHECK THE LOG FILES (NodeName_thread_log.log)
execute_commands_on_threads(slice.get_nodes(), commands)
print("Note this is executed on threads, check the log files for completion")

In [ ]:
#
# if you need to update already cloned repo
#
# update the code, compile and test
# note that most live tests only need the simplest URI - ejfats://token@ip:port/
# however the e2sar_reas_live_test requires data and sync addresses, and data address must
# be real (so we use loopback). Hence the long form of the URI for live tests 
# (other tests simply ignore the parts of the URI they don't need.)

commands = [
    f"cd E2SAR; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git pull origin {e2sar_branch}",
    f"BOOST_ROOT=/usr/local/ PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/ meson setup -Dpkg_config_path=/usr/local/lib/pkgconfig/:/usr/lib/lib64/pkgconfig/ --prefix {home_location}/e2sar-install build --wipe && sed -i 's/-std=c++11//g' build/build.ninja",
    f"PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/  meson compile -j 8 -C build",
#    f"cd E2SAR/build; PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/  meson test {e2sar_test_suite} --suite unit --timeout 0",
]
  
execute_commands_on_threads(slice.get_nodes(), commands)
print("Note this is executed on threads, check the log files for completion")

## Run Tests

### Run simple single-threaded performance test

Start Segmenter on Sender node and one Reassembler on Worker1 and test throughput **without a real load balancer**. Reassembler is told that LB header will be present and it ignores it.

In [ ]:
sender = list(filter(lambda n: n.get_name()[0:6] == "Sender", slice.get_nodes()))[0]
recver = list(filter(lambda n: n.get_name()[0:7] == "Worker1", slice.get_nodes()))[0]

sender_addr = sender.get_interface(network_name=make_net_name(sender.get_site())).get_ip_addr()
recver_addr = recver.get_interface(network_name=make_net_name(recver.get_site())).get_ip_addr()
print(f"Sender sending from {sender_addr}, receiver receiving on {recver_addr}")

In [ ]:
# for e2sar_perf only the data= part of the query is meaningful. sync= must be present but is ignored
# same for gRPC token, address and port (and lb id)
e2sarPerfURI = f"ejfat://useless@10.10.10.10:1234/lb/1?data={recver_addr}&sync=192.168.77.7:1234"
recverDuration = 30
mtu = 9000
rate = 10 # Gbps
length = 1000000 # event length in bytes
numEvents = 10000 # number of events to send
bufSize = 300 * 1024 * 1024 # 100MB send and receive buffers
recvThreads = 1 # makes no sense to use more than one in this scenario
coreList = "1" # space-separated list of cores
dequeThreads = 1 # threads that remove reassembled events off queue
useCoreList = False

if not useCoreList:
    # without thread affinity
    recv_command = f"cd E2SAR; PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/ ./build/bin/e2sar_perf -r -u '{e2sarPerfURI}' -d {recverDuration} -b {bufSize} --ip {recver_addr} --port 19522 --deq {dequeThreads} --threads {recvThreads}"
else:
    # with thread affinity
    recv_command = f"cd E2SAR; PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/ ./build/bin/e2sar_perf -r -u '{e2sarPerfURI}' -d {recverDuration} -b {bufSize} --ip {recver_addr} --port 19522 --deq {dequeThreads} --cores {coreList}"

send_command = f"cd E2SAR; PATH=$HOME/.local/bin:/usr/local/bin:$PATH LD_LIBRARY_PATH=/usr/local/lib/ ./build/bin/e2sar_perf -s -u '{e2sarPerfURI}' --mtu {mtu} --rate {rate} --length {length} -n {numEvents} -b {bufSize} --ip {sender_addr}"

# start the receiver for N seconds and log its output
print(f'Executing command {recv_command} on receiver')
recver.execute_thread(recv_command, output_file=f"{recver.get_name()}.perf.log")

# sleep 2 seconds to let receiver get going
time.sleep(5)

# start the sender in the foreground
print(f'Executing command {send_command} on sender')
stdout_send, stderr_send = sender.execute(send_command, output_file=f"{sender.get_name()}.perf.log")

print(f"Inspect {recver.get_name()}.perf.log file in your Jupyter container to see the results")

### Reserve the Load Balancer

In [ ]:
# ejfat admin URI is set in the preamble

lb_path = './E2SAR/build/bin'
ld_library_path = "LD_LIBRARY_PATH=/usr/local/lib"
bin_path = "PATH=./E2SAR/build/bin:$PATH"
# note we are forcing IPv4 here with -4 option - from FABRIC this is necessary
lbadm = f"{ld_library_path} {bin_path} lbadm -4"
e2sar_perf = f"{ld_library_path} {bin_path} e2sar_perf"


In [ ]:
# run an overview command to see what is reserved
# we use sender node but any node can be used for admin commands

command = f"{lbadm} --overview -u {ejfat_admin_uri}"

execute_commands(sender, [command])

In [ ]:
# Reserve the load balancer
lbname = 'e2sar-testlb'
duration = '02:00:00' # 2 hours

command = f"{lbadm} --reserve -l {lbname} -a '{sender_addr}' -d {duration} -u '{ejfat_admin_uri}' -e"
execute_commands(sender, [command])

In [ ]:
# copy the 'Updated URI after reserve with instance token' from the above result here:
instance_uri = 'ejfats://<replace with output from previous command>'

In [ ]:
# get the status of the reserved LB (as a check)
command = f"{lbadm} --status -u '{instance_uri}'"
execute_commands(sender, [command])

### Run a test with real load balancer and single sender node and single receiver node

In [ ]:
# select sender and receiver
sender = list(filter(lambda n: n.get_name()[0:6] == "Sender", slice.get_nodes()))[0]
recver = list(filter(lambda n: n.get_name()[0:7] == "Worker1", slice.get_nodes()))[0]

sender_addr = sender.get_interface(network_name=make_net_name(sender.get_site())).get_ip_addr()
recver_addr = recver.get_interface(network_name=make_net_name(recver.get_site())).get_ip_addr()
print(f"Sender sending from {sender_addr}, receiver receiving on {recver_addr}")

In [ ]:
# run sender and receiver through the reserved LB
recverDuration = 60
mtu = 9000
rate = 10 # Gbps
length = 1000000 # event length in bytes
numEvents = 20000 # number of events to send
bufSize = 300 * 1024 * 1024 # 100MB send and receive buffers
recvThreads = 6
coreList = "1 2 3 4 5 6"
dequeThreads = 1
useCoreList = False

# Some notable options:
# Sender:
# -z - send 0 event rate in Sync messages
# --seq - use sequential numbers as LB and Sync event numbers 
# Receiver:
# --deq - number of threads reading events off reassembly queue
# --cores - list of cores to bind receiver threads to

# Given that in FABRIC ejfat-lb.es.net resolves to IP6 first and gRPC C++ library doesn't
# offer granular control over which resolved address is used, we use -4 option together with
# --withcp to tell the code to use the IPv4 address, but this also disables cert validation.

send_command = f"{e2sar_perf} -s -u '{instance_uri}' --mtu {mtu} --rate {rate} --length {length} -n {numEvents} -b {bufSize} --ip {sender_addr} --withcp -4"

# we can receive with threads bound to cores (vCPUs) or not
if not useCoreList:
    recv_command = f"{e2sar_perf} -r -u '{instance_uri}' -d {recverDuration} -b {bufSize} --ip {recver_addr} --port 10000 --withcp -4 --threads {recvThreads}"
else:
    recv_command = f"{e2sar_perf} -r -u '{instance_uri}' -d {recverDuration} -b {bufSize} --ip {recver_addr} --port 10000 --withcp -4 --cores {coreList}"

# start the receiver for n seconds and log its output
print(f'Executing command {recv_command} on receiver')
recver.execute_thread(recv_command, output_file=f"{recver.get_name()}.perf.log")

# sleep 5 seconds to let receiver get going
time.sleep(6)

# start the sender in the foreground
print(f'Executing command {send_command} on sender')
stdout_send, stderr_send = sender.execute(send_command, output_file=f"{sender.get_name()}.perf.log")

print(f"Inspect {recver.get_name()}.perf.log file to see the results. It should indicate how many events were received and how many lost.")

### Run the test with a single sender node and multiple receiver nodes

In [ ]:
# select sender and receivers
sender = list(filter(lambda n: n.get_name()[0:6] == "Sender", slice.get_nodes()))[0]

worker_index = 1
recvers = list()
recver_addrs = list()
while True:
    matches = list(filter(lambda n: n.get_name()[0:7] == f"Worker{worker_index}", slice.get_nodes()))
    if len(matches) == 0:
        break
    recver = matches[0]
    recvers.append(recver)
    recver_addr = recver.get_interface(network_name=make_net_name(recver.get_site())).get_ip_addr()
    recver_addrs.append(recver_addr)
    worker_index += 1
    
sender_addr = sender.get_interface(network_name=make_net_name(sender.get_site())).get_ip_addr()
print(f"Sender sending from {sender_addr}, receivers receiving on:")
for recver, recver_addr in zip(recvers, recver_addrs):
    print("\t" + recver.get_name() + ": " + recver_addr)

In [ ]:
# run sender and receivers through the reserved LB
recverDuration = 60
mtu = 9000
rate = 20 # Gbps
length = 1000000 # event length in bytes
numEvents = 20000 # number of events to send
bufSize = 300 * 1024 * 1024 # 100MB send and receive buffers
recvThreads = 6 # on each receiver
numSocks = 4 # number send sockets in sender
startPort = 10000 # receiver starting port
dequeThreads = 1

# Some notable options:
# Sender:
# -z - send 0 event rate in Sync messages
# --seq - use sequential numbers as LB and Sync event numbers 
# Receiver:
# --deq - number of threads reading events off reassembly queue
# --cores - list of cores to bind receiver threads to

# Given that in FABRIC ejfat-lb.es.net resolves to IP6 first and gRPC C++ library doesn't
# offer granular control over which resolved address is used, we use -4 option together with
# --withcp to tell the code to use the IPv4 address, but this also disables cert validation.

send_command = f"{e2sar_perf} -s -u '{instance_uri}' --mtu {mtu} --rate {rate} --length {length} -n {numEvents} -b {bufSize} --ip {sender_addr} --sockets {numSocks} --withcp -4"

for recver, recver_addr in zip(recvers, recver_addrs):
    recv_command = f"{e2sar_perf} -r -u '{instance_uri}' -d {recverDuration} -b {bufSize} --ip {recver_addr} --port {startPort} --withcp -4 --threads {recvThreads} --deq {dequeThreads}"
    print(f'Executing command {recv_command} on receiver {recver.get_name()}')
    recver.execute_thread(recv_command, output_file=f"{recver.get_name()}.perf.log")

# sleep 5 seconds to let receivers get going
time.sleep(5)

# start the sender in the foreground
print(f'Executing command {send_command} on sender')
stdout_send, stderr_send = sender.execute(send_command, output_file=f"{sender.get_name()}.perf.log")

print(f"Inspect WorkerX.perf.log files to see the results. They should indicate how many events were received and how many lost.")

In [ ]:
# free the load balancer
command = f"{lbadm} --free -u '{instance_uri}'"

execute_commands(sender, [command])

## Manage the slice

### Extend by two weeks

In [ ]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

### Delete

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()